In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import numpy as np
import joblib

In [2]:
df=pd.read_csv("threat_prediction1.csv")
df=df.drop(columns=["Personnel_Drone_Control","Personnel_Signal_Intelligence"])
df

,Threat_ID,Threat_Type,Severity,Location_Type,Civilian_Count,Military_Assets,Personnel_Bomb_Disposal,Personnel_Engineering,Personnel_Surveillance,Personnel_Combat,Personnel_Weapons,Personnel_Intelligence,Personnel_Cyber,Personnel_Medical,Personnel_Tactics,Personnel_Strategy,Personnel_Recon,Personnel_Negotiation,Personnel_Sniper
0,10001,riots,2,public place,4342,7,0,0,6,0,14,14,0,0,12,16,0,0,0
1,10002,riots,5,public place,680,5,0,0,25,0,40,30,0,0,40,30,0,0,0
2,10003,cyber attack,5,data center,29,19,0,0,0,0,0,20,55,0,0,0,0,0,0
3,10004,terrorist attack,2,urban area,1752,15,8,0,0,22,20,0,0,0,8,14,0,0,4
4,10005,bomb threat,4,public place,4328,4,12,12,16,76,24,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,19996,cyber attack,5,data center,3516,7,0,0,0,0,0,50,65,0,0,0,0,0,0
9996,19997,natural disaster,3,forest,2579,3,0,0,9,54,0,0,0,15,0,0,0,0,0
9997,19998,hijack,2,metro station,1303,6,4,0,16,38,18,20,0,0,8,14,6,3,0
9998,19999,cyber attack,1,private organization,4385,19,0,0,0,0,0,4,10,0,0,0,0,0,0


In [3]:
x=df.iloc[:,1:6]
y=df.iloc[:,6:]

In [4]:
x

,Threat_Type,Severity,Location_Type,Civilian_Count,Military_Assets
0,riots,2,public place,4342,7
1,riots,5,public place,680,5
2,cyber attack,5,data center,29,19
3,terrorist attack,2,urban area,1752,15
4,bomb threat,4,public place,4328,4
...,...,...,...,...,...
9995,cyber attack,5,data center,3516,7
9996,natural disaster,3,forest,2579,3
9997,hijack,2,metro station,1303,6
9998,cyber attack,1,private organization,4385,19


In [5]:
y

,Personnel_Bomb_Disposal,Personnel_Engineering,Personnel_Surveillance,Personnel_Combat,Personnel_Weapons,Personnel_Intelligence,Personnel_Cyber,Personnel_Medical,Personnel_Tactics,Personnel_Strategy,Personnel_Recon,Personnel_Negotiation,Personnel_Sniper
0,0,0,6,0,14,14,0,0,12,16,0,0,0
1,0,0,25,0,40,30,0,0,40,30,0,0,0
2,0,0,0,0,0,20,55,0,0,0,0,0,0
3,8,0,0,22,20,0,0,0,8,14,0,0,4
4,12,12,16,76,24,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,50,65,0,0,0,0,0,0
9996,0,0,9,54,0,0,0,15,0,0,0,0,0
9997,4,0,16,38,18,20,0,0,8,14,6,3,0
9998,0,0,0,0,0,4,10,0,0,0,0,0,0


In [6]:
scaler=StandardScaler()
encoder=OneHotEncoder()

In [7]:
txt=["Threat_Type","Location_Type"]
num=["Severity","Civilian_Count","Military_Assets"]

In [8]:
processing=ColumnTransformer(transformers=[
    ("text",encoder,txt),
    ("num",scaler,num)
])

In [9]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.1)

In [78]:

pipline1=Pipeline(steps=[
    ("preprocessor",processing),
    ("model",MultiOutputRegressor(XGBRegressor(n_estimators=40, learning_rate=0.1)))
])

In [79]:
pipline1.fit(xtrain,ytrain)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text', OneHotEncoder(),
                                                  ['Threat_Type',
                                                   'Location_Type']),
                                                 ('num', StandardScaler(),
                                                  ['Severity', 'Civilian_Count',
                                                   'Military_Assets'])])),
                ('model',
                 MultiOutputRegressor(estimator=XGBRegressor(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_b...
                                                             grow_policy=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.1,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=40,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=None, ...)))])

In [85]:
joblib.dump(pipline1,'thread_model.pkl')

['thread_model.pkl']

In [80]:
predict1=pipline1.predict(xtest)

In [81]:
predict1

array([[5.6835908e-02, 1.5338235e-02, 5.5983062e+00, ..., 3.4383857e+00,
        5.4589533e-03, 9.4076963e-03],
       [8.8288975e+00, 1.5338235e-02, 1.6605644e+01, ..., 1.0413242e+01,
        2.4584849e+00, 9.4076963e-03],
       [5.6835908e-02, 1.5338235e-02, 2.6838917e+01, ..., 4.4174507e-02,
        5.4589533e-03, 9.4076963e-03],
       ...,
       [3.1724784e+00, 1.5338235e-02, 5.8461509e+00, ..., 3.1565793e+00,
        2.5314333e+00, 9.4076963e-03],
       [5.6835908e-02, 1.5338235e-02, 1.7511651e-01, ..., 4.4174507e-02,
        5.4589533e-03, 9.4076963e-03],
       [6.2085867e+00, 1.5338235e-02, 1.0760402e+01, ..., 6.4805989e+00,
        2.2824917e+00, 9.4076963e-03]], dtype=float32)

In [82]:
arr1=np.round(predict1).astype(int)
arr1

array([[ 0,  0,  6, ...,  3,  0,  0],
       [ 9,  0, 17, ..., 10,  2,  0],
       [ 0,  0, 27, ...,  0,  0,  0],
       ...,
       [ 3,  0,  6, ...,  3,  3,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 6,  0, 11, ...,  6,  2,  0]])

In [83]:
accuracy1=r2_score(arr1,ytest)
accuracy1

0.8729411459957805

In [84]:
mse1=mean_squared_error(arr1,ytest)
mse1

14.555923076923078

In [19]:
new_data = pd.DataFrame([{
    'Threat_Type': 'cyber attack',
    'Severity': 5,
    'Location_Type': 'public place',
    'Civilian_Count': 5,
    'Military_Assets': 3
}])

test1=pipline1.predict(new_data)
testing1=np.round(test1).astype(int)

In [20]:
t_1=testing1.reshape(-1)
t_1[t_1<0]=0
t_1

array([ 0,  0,  0,  0,  0, 28, 49,  0,  0,  0,  0,  0,  0])